<a href="https://colab.research.google.com/github/42-HackKosice/3B-ReadDataFrom.tiff/blob/main/EllipseFitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3B - Ellipse fitting

In [150]:
#This program should help us with reading the images and printing the output value to .csv.
#Created by:")
#   Timotej Breznický - 3rd grade
#   Matej Mazúr - 3rd grade
#   Marko Poľanský - 3rd grade
# From High school of electrical engineering in Prešov, Slovakia

Imports

In [151]:
import numpy as np
import cv2
import os
from google.colab.patches import cv2_imshow
import time

Functions

In [152]:
def findLargestContour(edgeImg):
  contours, hierarchy = cv2.findContours(
    edgeImg,
    cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE
  )
  # From among them, find the contours with large surface area.
  contoursWithArea = []
  for contour in contours:
    area = cv2.contourArea(contour)
    contoursWithArea.append([contour, area])
  contoursWithArea.sort(key=lambda tupl: tupl[1], reverse=True)
  largestContour = contoursWithArea[0][0]
  return largestContour

def GetContureOfImage(imgPath):
  src = cv2.imread(imgPath, 1)
  blurred = cv2.GaussianBlur(src, (5, 5), 0)

  blurred_float = blurred.astype(np.float32) / 255.0
  # download model from https://github.com/opencv/opencv_extra/blob/5e3a56880fb115e757855f8e01e744c154791144/testdata/cv/ximgproc/model.yml.gz
  edgeDetector = cv2.ximgproc.createStructuredEdgeDetection("/content/model.yml")
  edges = edgeDetector.detectEdges(blurred_float) * 255.0
  edgeDetector.clear()

  contour = findLargestContour(edges_8u)

  #Draw Ellips to the original image
  ellipse = cv2.fitEllipse(contour)

  return ellipse

Final Program

In [156]:
path = r"/content/Img"
outputFile = "filename,ellipse_center_x,ellipse_center_y,ellipse_majoraxis,ellipse_minoraxis,ellipse_angle,elapsed_time_ms\n"
with os.scandir(path) as dirs:
  for entry in dirs:
    if(entry.name[-5:] == ".tiff"):
      ActualTime = time.time()
      imgpath = "/content/Img/"+entry.name
      line = entry.name + ","
      try:
        _result = GetContureOfImage(imgpath)
        line += str(_result).replace("(","").replace(")","").replace(" ","")
      except:
        line = entry.name + ",,,,,,"
      outputFile += line + "," + str(round((time.time()-ActualTime)*1000)) + "\n"

In [157]:
#print(outputFile)
fo = open("EllipseFittingOutput.csv", "w")
fo.write(outputFile)
fo.close()